In [1]:
import cv2
import torch
from ultralytics import YOLO

In [2]:
def natural_sort_key(filename):
    """Extract numerical values from filenames for correct sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

In [3]:
def detect_objects(model, image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Perform object detection
    return results, image


In [4]:
def get_ceiling_floor_coordinates(results, floor_model):
    ceiling_y = None  # Initialize
    floor_y = None  # Initialize
    best_ceiling = None
    best_floor = None

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coords
            confidence = float(box.conf[0].cpu().numpy())  # Confidence score
            class_id = int(box.cls[0].cpu().numpy())  # Class ID
            
            class_name = floor_model.names.get(class_id, "Unknown")

            if class_name == "ceiling":
                if best_ceiling is None or confidence > best_ceiling[1]:
                    best_ceiling = ((x1, y1, x2, y2), confidence)

            if class_name == "floor":
                if best_floor is None or confidence > best_floor[1]:
                    best_floor = ((x1, y1, x2, y2), confidence)

    ceiling_mid, floor_mid = None, None  

    if best_ceiling:
        (_, ceiling_y1, _, ceiling_y2), _ = best_ceiling
        ceiling_y = (ceiling_y1 + ceiling_y2)// 2  # Midpoint

    if best_floor:
        (_, floor_y1, _, floor_y2), _ = best_floor
        floor_y = (floor_y1 + floor_y2)// 2  # Midpoint

    return ceiling_y, floor_y


In [5]:
def get_floor_coordinates(results):
    """Extract the top Y-coordinate and bounding box of the floor"""
    floor_y_top = None
    floor_box = None
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 1:  # Assuming class 1 is "floor"
                floor_y_top = y_min  # Top Y-coordinate of floor
                floor_box = (x_min, y_min, x_max, y_max)
    
    return floor_y_top, floor_box

In [6]:
def get_propeller_coordinates(results):
    """Extract the bottom Y-coordinates and bounding boxes of propellers"""
    propeller_boxes = []  # Store bounding boxes of detected propellers
    propeller_y_bottoms = []  # Store Y-bottom of propellers
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 0:  # Assuming class 0 is "propeller"
                propeller_y_bottoms.append(y_max)  # Store bottom Y-coordinate
                propeller_boxes.append((x_min, y_min, x_max, y_max))
    
    return propeller_y_bottoms, propeller_boxes

In [7]:
def detect_height_bottle(image_path, model):
    """Detect Coca-Cola bottles and return their bounding box height in pixels."""
    image = cv2.imread(image_path)
    results = model(image)
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
            height_pixels = y2 - y1  # Bottle height in pixels
            real_height_cm = 26  # Known height of Coca-Cola bottle
            
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            return height_pixels, estimated_real_height

    return None, None  # If no bottle is detected

In [8]:
def calculate_height(ceiling_y, floor_y, calibration_obj):
    """Calculate ceiling height using a calibration object (Coca-Cola bottle)."""
    if ceiling_y is None or floor_y is None:
        print("Error: Ceiling or Floor not detected!")
        return None  # Avoid TypeError

    pixel_height = floor_y - ceiling_y

    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        pixel_to_cm_ratio = real_height / obj_pixel_height
        return pixel_height * pixel_to_cm_ratio
    
    return pixel_height  # Return pixel height if no calibration object


In [9]:
def draw_detections(image, propeller_boxes, floor_box, calibration_obj, floor_results):
    best_ceiling = None
    best_floor = None  # Ensure variables are initialized

    for i, (x1, y1, x2, y2) in enumerate(propeller_boxes):  # Unpacking tuple directly
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # Ensure integer values
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Calculate height from floor to propeller
        pixel_height = y1 - floor_box[1]
        real_height = calculate_height(y1, floor_box[1], calibration_obj)
        height_text = f"Height: {real_height:.2f} cm"
        
        # Draw text label
        cv2.putText(image, height_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Label the calibration object
    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        cal_text = f"Calibration: {real_height:.2f} cm ({obj_pixel_height} px)"
        cv2.putText(image, cal_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    # Process results
    for result in floor_results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coordinates
            confidence = float(box.conf[0].cpu().numpy())  # Confidence score
            class_id = int(box.cls[0].cpu().numpy())  # Class ID

            # Get class name correctly
            class_name = floor_model.names.get(class_id, "Unknown")  # Ensure we get the correct class label

            # Store the most confident "ceiling" and "floor" detection
            if class_name == "ceiling":
                if best_ceiling is None or confidence > best_ceiling[1]:
                    best_ceiling = ((x1, y1, x2, y2), confidence, class_name)

            if class_name == "floor":
                if best_floor is None or confidence > best_floor[1]:
                    best_floor = ((x1, y1, x2, y2), confidence, class_name)

    # Draw only the best detections on the image
    for best in [best_ceiling, best_floor]:
        if best is not None:  # Ensure best_ceiling and best_floor are not None
            (x1, y1, x2, y2), confidence, class_name = best
            label = f'{class_name}: {confidence:.2f}'
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return image


In [10]:
def draw_annotations(image, coke_boxes, ceiling_y, floor_y, pixel_to_cm_ratio=None):
    """Draws bounding boxes, vertical line, and labels on the image."""
    annotated_image = image.copy()
    
    # Draw Coca-Cola bounding boxes
    for (x1, y1, x2, y2, height_pixels) in coke_boxes:
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
        cv2.putText(annotated_image, f"{height_pixels}px", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw vertical line from floor to ceiling
    if ceiling_y is not None and floor_y is not None:
        mid_x = image.shape[1] // 2  # Center x-coordinate
        cv2.line(annotated_image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 0, 255), 2)  # Red line
        
        # Calculate real-world ceiling height if calibration object exists
        pixel_height = floor_y - ceiling_y
        if pixel_to_cm_ratio:
            real_ceiling_height = pixel_height * pixel_to_cm_ratio
            height_text = f"Height: {real_ceiling_height:.2f} cm"
        else:
            height_text = f"Height: {pixel_height} pixels"

        # Add text label
        cv2.putText(annotated_image, height_text, (mid_x + 10, (ceiling_y + floor_y) // 2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    return annotated_image

In [11]:
# # Define folders
# image_folder = "Input_test_folder(3m approx)"
# output_folder = "output_test_folder(3m approx)"
# os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists

# coke_model_path = "best (8).pt"
# ceiling_floor_model_path = "best (5).pt"
# coke_model = YOLO(coke_model_path)  # Load Coca-Cola bottle detection model
# ceiling_floor_model = YOLO(ceiling_floor_model_path)
# image_heights = {}  # Declare outside to store all images' heights

# for image_name in os.listdir(image_folder):
#     if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
#         image_path = os.path.join(image_folder, image_name)

#         # Detect Coca-Cola bottle height
#         height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)

#         if height_pixels is None:
#             print(f"Skipping {image_name}: No Coca-Cola bottle detected.")
#             continue

#         print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
#         print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")

#         # Detect ceiling and floor
#         results = ceiling_floor_model(image_path)
#         ceiling_y, floor_y = get_ceiling_floor_coordinates(results)

#         if ceiling_y is None or floor_y is None:
#             print(f"Skipping {image_name}: Ceiling or floor not detected.")
#             continue

#         # Calculate ceiling height using Coca-Cola bottle as a calibration object
#         calibration_obj = (estimated_real_height, height_pixels)
#         ceiling_height = calculate_height(ceiling_y, floor_y, calibration_obj)

#         print(f"Ceiling Height: {ceiling_height:.2f} cm")

#         # image_heights = {}

#         image_heights[image_name] = ceiling_height  # Dynamically update dictionary
#         image = cv2.imread(image_path)
        

#         # Detect objects
#         coke_boxes = detect_coke_bottle(image)
#         ceiling_y, floor_y = detect_ceiling_floor(image)
        
#         # Calibration: Use Coca-Cola bottle to calculate pixel-to-cm ratio
#         if coke_boxes:
#             real_coke_height_cm = 24
#             _, _, _, _, height_pixels = coke_boxes[0]  # First detected bottle
#             pixel_to_cm_ratio = real_coke_height_cm / height_pixels
#         else:
#             pixel_to_cm_ratio = None

#         # Draw results
#         annotated_image = draw_annotations(image, coke_boxes, ceiling_y, floor_y, pixel_to_cm_ratio)

#         # calculated_height = compute_height(image_name)  # Replace with your actual function
#         # image_heights[image_name] = calculated_height  # Dynamically update dictionary
#         # Save annotated image
#         output_path = os.path.join(output_folder, f"annotated_{image_name}")
#         cv2.imwrite(output_path, annotated_image)

# print("Processing complete! Check the 'output_images' folder.")

# This code able to calculate the height of floor to ceiling .

# import os
# image_folder = "Input_test_folder(3m approx)"
# output_folder = "Output_test_folder_propeller"
# model_floor_path = "best (5).pt"
# model_propeller_path = "best (9).pt"
# coke_model_path = "best (8).pt"
# coke_model = YOLO(coke_model_path)

# os.makedirs(output_folder, exist_ok=True)
# floor_model = load_model(model_floor_path)
# propeller_model = load_model(model_propeller_path)
# for image_name in os.listdir(image_folder):
#     image_path = os.path.join(image_folder, image_name)

    
    
#     height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)

#     if height_pixels is None:
#         print(f"Skipping {image_name}: No Coca-Cola bottle detected.")
#         continue

#     print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
#     print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")
    
#     calibration_obj = (estimated_real_height,height_pixels)
    
#     floor_results, image = detect_objects(floor_model, image_path)
#     floor_y_top, floor_box = get_floor_coordinates(floor_results)
#     propeller_results, _ = detect_objects(propeller_model, image_path)
#     propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)
#     if not propeller_boxes or floor_y_top is None:
#         print(f"No propellers or floor detected in {image_path}")
#         continue
#     print(f"\nImage: {image_path}")
#     for i, propeller_y_bottom in enumerate(propeller_y_bottoms):
#         real_height = calculate_height(propeller_y_bottom, floor_y_top, calibration_obj)
#         print(f"  Propeller {i+1}: {real_height:.2f} cm from floor")
#     output_image = draw_detections(image, propeller_boxes, floor_box, calibration_obj)
#     output_path = os.path.join(output_folder, image_name)
#     cv2.imwrite(output_path, output_image)
#     print(f"Output saved at: {output_path}")


In [12]:
# import os
# import re

# def natural_sort_key(filename):
#     """Extract numerical values from filenames for correct sorting."""
#     return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

# # Define folders
# image_folder = "Input_test_folder(3m approx)"
# output_folder = "Output_test_folder_combined"
# os.makedirs(output_folder, exist_ok=True)

# # Load models
# coke_model = YOLO("best (8).pt")
# floor_model = YOLO("best (5).pt")
# propeller_model = YOLO("best (9).pt")

# image_heights = {}  # Declare outside to store all images' heights

# # Store image results
# image_results = []

# # Get and sort image filenames using natural sorting
# image_files = sorted(
#     [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
#     key=natural_sort_key
# )

# for image_name in image_files:  # Process images in correct order
#     image_path = os.path.join(image_folder, image_name)
    
#     # Detect Coca-Cola bottle height (for calibration)
#     height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)
#     if height_pixels is None:
#         print(f"Skipping {image_name}: No Coca-Cola bottle detected.")
#         calibration_obj = None
#     else:
#         calibration_obj = (estimated_real_height, height_pixels)
    
#     # Detect ceiling and floor
#     floor_results, image = detect_objects(floor_model, image_path)
#     ceiling_y, floor_y = get_ceiling_floor_coordinates(floor_results)

#     if ceiling_y is None or floor_y is None:
#         ceiling_height = "XXXXX"
#     elif calibration_obj:
#         ceiling_height = f"{calculate_height(ceiling_y, floor_y, calibration_obj):.2f} cm"
#     else:
#         ceiling_height = "XXXXX"

#     print(f"Ceiling Height for {image_name}: {ceiling_height}")

#     # image_heights = {}

#     image_heights[image_name] = ceiling_height  # Dynamically update dictionary
#     image = cv2.imread(image_path)

#     # Detect propellers (AC)
#     propeller_results, _ = detect_objects(propeller_model, image_path)
#     propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)
    
#     if propeller_boxes and calibration_obj:
#         propeller_heights = [f"{calculate_height(y, floor_y, calibration_obj):.2f} cm" for y in propeller_y_bottoms]
#     else:
#         propeller_heights = []

#     # Format AC height output
#     ac_height = ", ".join(propeller_heights) if propeller_heights else "YYYY"
#     print(f"AC Height for {image_name}: {ac_height}")

#     # Draw detections
#     if calibration_obj:
#         output_image = draw_detections(image, propeller_boxes, (floor_y, floor_y), calibration_obj)
#         output_path = os.path.join(output_folder, image_name)
#         cv2.imwrite(output_path, output_image)
#         print(f"Output saved at: {output_path}")

#     # Store results in required format
#     image_results.append(f"{image_name} | ID photo 123456 | Ceiling height {ceiling_height} | AC Height {ac_height}")

# # Print final results
# print("\nFinal Results:")
# for result in image_results:
#     print(result)

In [13]:
import os
import re
import cv2
import numpy as np

def natural_sort_key(filename):
    """Extract numerical values from filenames for correct sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

# Define folders
image_folder = "Input_test_folder(3m approx)"
output_folder = "OUTPUT"
os.makedirs(output_folder, exist_ok=True)

# Load models
coke_model = YOLO("best (8).pt")
floor_model = YOLO("best (5).pt")
propeller_model = YOLO("best (9).pt")

image_heights = {}  # Store detected ceiling heights
image_results = []

# Get and sort image filenames
image_files = sorted(
    [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
    key=natural_sort_key
)

i = 0
for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    
    # Detect Coca-Cola bottle height (for calibration)
    height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)
    calibration_obj = (estimated_real_height, height_pixels) if height_pixels else None
    
    # Detect ceiling and floor
    floor_results, image = detect_objects(floor_model, image_path)
    # Perform inference
        

    # Store the best detection for ceiling and floor
    # best_ceiling = None
    # best_floor = None

    # # Process results
    # for result in floor_results:
    #     for box in result.boxes:
    #         x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coordinates
    #         confidence = float(box.conf[0].cpu().numpy())  # Confidence score
    #         class_id = int(box.cls[0].cpu().numpy())  # Class ID
                
    #         # Get class name correctly
    #         class_name = floor_model.names.get(class_id, "Unknown")  # Ensure we get the correct class label

    #         # Store the most confident "ceiling" and "floor" detection
    #         if class_name == "ceiling":
    #             if best_ceiling is None or confidence > best_ceiling[1]:
    #                 best_ceiling = ((x1, y1, x2, y2), confidence, class_name)

    #         if class_name == "floor":
    #             if best_floor is None or confidence > best_floor[1]:
    #                 best_floor = ((x1, y1, x2, y2), confidence, class_name)

    #     # Draw only the best detections on the image
    # for best in [best_ceiling, best_floor]:
    #     if best is not None:
    #         (x1, y1, x2, y2), confidence, class_name = best
    #         label = f'{class_name}: {confidence:.2f}'
    #         cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    #         cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    
    ceiling_y,floor_y = get_ceiling_floor_coordinates(floor_results,floor_model)
    
    if ceiling_y is None or floor_y is None:
        ceiling_height = "XXXXX"
    elif calibration_obj:
        ceiling_height = calculate_height(ceiling_y, floor_y, calibration_obj)
    else:
        ceiling_height = "XXXXX"
    
    print(f"Ceiling Height for {image_name}: {ceiling_height}")
    
    if ceiling_height != "XXXXX":
        image_heights[image_name] = ceiling_height
    
    # Read the image
    image = cv2.imread(image_path)
    
    # Detect propellers (AC)
    propeller_results, _ = detect_objects(propeller_model, image_path)
    propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)
    
    if propeller_boxes and calibration_obj:
        propeller_heights = [f"{calculate_height(y, floor_y, calibration_obj):.2f} cm" for y in propeller_y_bottoms]
    else:
        propeller_heights = []
    
    ac_height = ", ".join(propeller_heights) if propeller_heights else "YYYY"
    print(f"AC Height for {image_name}: {ac_height}")
    
    
    # Draw vertical reference line
    if ceiling_y is not None and floor_y is not None and ceiling_height != "XXXXX":
        mid_x = image.shape[1] // 2
        cv2.line(image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 255, 0), 3)
        text_position = (mid_x + 10, (ceiling_y + floor_y) // 2)
        cv2.putText(image, f"{ceiling_height:.2f} cm", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Draw propeller detections
    if calibration_obj:
        output_image = draw_detections(image, propeller_boxes, (floor_y, floor_y), calibration_obj,floor_results)
        output_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_path, output_image)
        print(f"Output saved at: {output_path}")
    
    # Store results
    image_results.append(f"Photo {i+1} | ID {image_name} | Ceiling height {ceiling_height} | AC Height {ac_height}")
    i += 1

# Print final results
print("\nFinal Results:")
for result in image_results:
    print(result)



0: 800x608 1 750ml coke, 436.0ms
Speed: 4.2ms preprocess, 436.0ms inference, 0.8ms postprocess per image at shape (1, 3, 800, 608)

0: 640x480 2 ceilings, 2 floors, 4 wallss, 334.6ms
Speed: 2.5ms preprocess, 334.6ms inference, 11.7ms postprocess per image at shape (1, 3, 640, 480)
Ceiling Height for image1.jpeg: 307.62640380859375

0: 800x608 (no detections), 343.8ms
Speed: 5.3ms preprocess, 343.8ms inference, 0.4ms postprocess per image at shape (1, 3, 800, 608)
AC Height for image1.jpeg: YYYY
Output saved at: OUTPUT/image1.jpeg

0: 800x608 1 750ml coke, 343.6ms
Speed: 4.1ms preprocess, 343.6ms inference, 0.8ms postprocess per image at shape (1, 3, 800, 608)

0: 640x480 5 floors, 2 wallss, 324.2ms
Speed: 2.7ms preprocess, 324.2ms inference, 9.9ms postprocess per image at shape (1, 3, 640, 480)
Ceiling Height for image2.jpeg: XXXXX

0: 800x608 (no detections), 345.2ms
Speed: 4.0ms preprocess, 345.2ms inference, 0.4ms postprocess per image at shape (1, 3, 800, 608)
AC Height for image2

In [14]:
# import os
# import re

# def natural_sort_key(filename):
#     """Extract numerical values from filenames for correct sorting."""
#     return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

# # Define folders
# image_folder = "Input_test_folder(3m approx)"
# output_folder = "Output_test_folder_combined"
# os.makedirs(output_folder, exist_ok=True)

# # Load models
# coke_model = YOLO("best (8).pt")
# floor_model = YOLO("best (5).pt")
# propeller_model = YOLO("best (9).pt")

# image_heights = {}  # Declare outside to store all images' heights

# # Store image results
# image_results = []

# # Get and sort image filenames using natural sorting
# image_files = sorted(
#     [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
#     key=natural_sort_key
# )

# for image_name in image_files:  # Process images in correct order
#     image_path = os.path.join(image_folder, image_name)
    
#     # Detect Coca-Cola bottle height (for calibration)
#     height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)
#     if height_pixels is None:
#         print(f"Skipping {image_name}: No Coca-Cola bottle detected.")
#         calibration_obj = None
#     else:
#         calibration_obj = (estimated_real_height, height_pixels)
    
#     # Detect ceiling and floor
#     floor_results, image = detect_objects(floor_model, image_path)
#     ceiling_y, floor_y = get_ceiling_floor_coordinates(floor_results)

#     if ceiling_y is None or floor_y is None:
#         ceiling_height = "XXXXX"
#     elif calibration_obj:
#         ceiling_height = calculate_height(ceiling_y, floor_y, calibration_obj)
#     else:
#         ceiling_height = "XXXXX"

#     print(f"Ceiling Height for {image_name}: {ceiling_height}")

#     # Store ceiling height only if it's numeric
#     if ceiling_height != "XXXXX":
#         image_heights[image_name] = ceiling_height  # Convert to float for consistency


#     # image_heights = {}

#     # image_heights[image_name] = ceiling_height  # Dynamically update dictionary
#     image = cv2.imread(image_path)

#     # Detect propellers (AC)
#     propeller_results, _ = detect_objects(propeller_model, image_path)
#     propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)
    
#     if propeller_boxes and calibration_obj:
#         propeller_heights = [f"{calculate_height(y, floor_y, calibration_obj):.2f} cm" for y in propeller_y_bottoms]
#     else:
#         propeller_heights = []

#     # Format AC height output
#     ac_height = ", ".join(propeller_heights) if propeller_heights else "YYYY"
#     print(f"AC Height for {image_name}: {ac_height}")

#     # Draw detections
#     if calibration_obj:
#         output_image = draw_detections(image, propeller_boxes, (floor_y, floor_y), calibration_obj)
#         output_path = os.path.join(output_folder, image_name)
#         cv2.imwrite(output_path, output_image)
#         print(f"Output saved at: {output_path}")

#     # Store results in required format
#     image_results.append(f"{image_name} | ID photo 123456 | Ceiling height {ceiling_height} | AC Height {ac_height}")

# # Print final results
# print("\nFinal Results:")
# for result in image_results:
#     print(result)

In [15]:
import numpy as np

# Predicted height (assumed constant for all images as per your requirement)
actual_height = 340# cm

# Extract actual heights from the provided results
# image_heights = {
#     "image2.jpeg": 197.32,
#     "image5.jpeg": 210.09,
#     "image4.jpeg": 196.10,
#     "image1.jpeg": 120.91,
# }

# List to store actual heights dynamically
predicted_heights = []
real_heights = []

# Fetch actual heights using a loop
# Convert actual heights to a NumPy array
predicted_heights = np.array(list(image_heights.values()))

# Create an array of the actual height for comparison
real_heights = np.full_like(predicted_heights, actual_height)

# Convert to NumPy arrays for calculations
# actual_heights = np.array(actual_heights)
# predicted_heights = np.full_like(actual_heights, predicted_height)

# Compute evaluation metrics
mae = np.mean(np.abs(real_heights - predicted_heights))  # Mean Absolute Error
mse = np.mean((real_heights - predicted_heights) ** 2)  # Mean Squared Error
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Print results
formatted_heights = ", ".join(f"{h:.2f}" for h in predicted_heights)
print(f"Predicted Heights: [{formatted_heights}]")
print(f"MAE: {mae:.2f} cm")
print(f"MSE: {mse:.2f} cm²")
print(f"RMSE: {rmse:.2f} cm")

Predicted Heights: [307.63, 304.52, 328.19, 368.68, 351.96, 321.48, 298.92, 379.23, 314.87, 299.76, 300.99, 322.66, 305.17, 285.68, 264.61, 289.51, 273.51, 323.06, 272.63, 274.91, 294.55, 301.83, 315.21, 321.85, 249.33, 245.54, 290.07, 343.80, 252.31, 338.03, 294.54, 344.35, 361.76, 267.81, 326.20, 314.78, 328.73, 313.20, 262.62, 309.03, 330.60, 278.74, 251.77, 337.99, 201.95, 257.33]
MAE: 42.12 cm
MSE: 2666.52 cm²
RMSE: 51.64 cm


In [16]:
# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((real_heights - predicted_heights) / real_heights)) * 100

# Calculate Accuracy
accuracy = 100 - mape

# Print results
print(f"MAPE: {mape:.2f}%")
print(f"Accuracy: {accuracy:.2f}%") 

MAPE: 12.39%
Accuracy: 87.61%
